In [1]:
from paracalc_impl_1d import Para1D

import numpy as np

In [2]:
para = Para1D(N=1024)
safe_freq = int(np.max(np.abs(para.freq))) - 1
print('Safe operations with frequency <= ', safe_freq)

Safe operations with frequency <=  511


In [6]:
# util functions

def random_example(para, max_freq, seed=0):
    np.random.seed(seed)
    f = np.random.randn(para.N) + 1j*np.random.randn(para.N)
    f[np.abs(para.freq) > max_freq] = 0
    return f

def map_freq(N, M, x):
    # the length of x should be 4*M+2
    f = np.zeros(N, dtype=complex)
    f[0] = x[0] + 1j*x[1]
    f[1:M+1] = x[2:M+2] + 1j*x[M+2:2*M+2]
    f[-1:-M-1:-1] = x[2*M+2:3*M+2] + 1j*x[3*M+2:]
    return f

$$\|f\|_{A^0} \leq\frac{1}{1-2^{-s}} \|f\|_{W^s}$$
The constant is optimal.

In [7]:
# test with random examples
num_exam = 10000
s = 1.0

r_max = 0.0
for i in range(num_exam):
    f = random_example(para, 256)
    r = para.A0_norm(f) / para.Ws_norm(f, s)
    r_max = max(r_max, r)
print('Max ratio: ', r)
print('Theoretical bound: ', 1.0/(1.0-2**(-s)))

Max ratio:  0.007948869615928224
Theoretical bound:  2.0


In [9]:
# test with optimization

np.random.seed(0)
for s in [1.0, 2.0, 2.5]:
    print('s = ', s)
    x0 = np.random.randn(513*2)

    def F_obj(x):
        f = map_freq(para.N, 256, x)
        r = para.A0_norm(f) / para.Ws_norm(f, s)
        return -r

    from scipy.optimize import minimize
    res = minimize(
        F_obj,
        x0,
        method="Powell",
        options={"maxiter": 10000, "disp": True}
    )
    print('Max ratio: ', -res.fun)
    print('Theoretical bound: ', 1.0/(1.0-2**(-s)))

s =  1.0
Optimization terminated successfully.
         Current function value: -1.996094
         Iterations: 2
         Function evaluations: 111929
Max ratio:  1.996093750000001
Theoretical bound:  2.0
s =  2.0
Optimization terminated successfully.
         Current function value: -1.333328
         Iterations: 2
         Function evaluations: 107494
Max ratio:  1.3333282470703132
Theoretical bound:  1.3333333333333333
s =  2.5
Optimization terminated successfully.
         Current function value: -1.214737
         Iterations: 2
         Function evaluations: 103129
Max ratio:  1.2147370290652089
Theoretical bound:  1.214737233854593


In [10]:
# test with optimal example
s = 1.0
f = np.zeros(para.N)
f[np.abs(para.freq) <= 1] = 1.0/3
for j in range(1, 9):
    f[(np.abs(para.freq) > 2**(j-1)) & (np.abs(para.freq) <= 2**j)] = 2**(-j*s-j)
    
r = para.A0_norm(f) / para.Ws_norm(f, s)
print('Ratio: ', r)
print('Theoretical bound: ', 1.0/(1.0-2**(-s)))

Ratio:  1.9960937499999998
Theoretical bound:  2.0


Paraproduct decomposition
$$
au=T_au+T_ua+R_{\rm PM}(a,u),
\quad R_{\rm PM}(a,u)=\sum_{j\geq0}\Delta_ja\cdot\Delta_ju,
$$

In [11]:
a = random_example(para, 192)
u = random_example(para, 192)

pd = para.product(a, u)
Ta_u = para.paraproduct(a, u)
Tu_a = para.paraproduct(u, a)

R_PM = np.zeros(para.N, dtype=complex)
for j in range(10):
    Rj = para.product(para.Delta_j(a, j), para.Delta_j(u, j))
    R_PM += Rj

np.max(np.abs(pd - Ta_u - Tu_a - R_PM))

4.263256414560601e-14

$$
\|T_au\|_{W^s}\leq \frac{2^{s}}{1-2^{-s}}\|a\|_{A^0}\|u\|_{W^s}.
$$

In [17]:
# test with optimization

np.random.seed(1)
for s in [1.0, 2.0, 2.5, 4.0]:
    print('s = ', s)
    x0 = np.random.randn(1028)

    def F_obj(x):
        a = map_freq(para.N, 128, x[:514])
        u = map_freq(para.N, 128, x[514:])
        Ta_u = para.paraproduct(a, u)
        r = para.Ws_norm(Ta_u, s) / para.A0_norm(a) / para.Ws_norm(u, s)
        return -r

    from scipy.optimize import minimize
    res = minimize(
        F_obj,
        x0,
        method="Powell",                # or "Nelder-Mead"
        options={"maxiter": 10000, "maxfev":200000, "disp": True}
    )
    print('Max ratio: ', -res.fun)
    print('Theoretical bound: ', 2**s/(1.0-2**(-s)))

s =  1.0
Optimization terminated successfully.
         Current function value: -2.000000
         Iterations: 2
         Function evaluations: 20658
Max ratio:  1.999999999999075
Theoretical bound:  4.0
s =  2.0
         Current function value: -4.655762
         Iterations: 11
         Function evaluations: 200000
Max ratio:  4.655761927294543
Theoretical bound:  5.333333333333333
s =  2.5
         Current function value: -6.364649
         Iterations: 9
         Function evaluations: 200000
Max ratio:  6.364648863510526
Theoretical bound:  6.871591483346974
s =  4.0
         Current function value: -16.765290
         Iterations: 10
         Function evaluations: 200000
Max ratio:  16.765290272927192
Theoretical bound:  17.066666666666666


$$
\|R_{\rm PM}(a,u)\|_{W^{s+r}}
\leq\frac{2^{r+s}}{1-2^{-(r+s)}}
\|a\|_{W^r}\|u\|_{W^s}.
$$

In [18]:
# test with optimization

np.random.seed(0)
for s in [1.0, 2.0, 2.5, 4.0]:
    s_ = s - 1
    print(f'(s, r) = ({s}, {s_})')
    x0 = np.random.randn(1028)

    def F_obj(x):
        a = map_freq(para.N, 128, x[:514])
        u = map_freq(para.N, 128, x[514:])
        Ta_u = para.paraproduct(a, u)
        Tu_a = para.paraproduct(u, a)
        R_pm = para.product(a, u) - Ta_u - Tu_a
        r = para.Ws_norm(R_pm, s+s_) / para.Ws_norm(a, s_) / para.Ws_norm(u, s)
        return -r

    from scipy.optimize import minimize
    res = minimize(
        F_obj,
        x0,
        method="Powell",                # or "Nelder-Mead"
        options={"maxiter": 10000, "maxfev":200000, "disp": True}
    )
    print('Max ratio: ', -res.fun)
    print('Theoretical bound: ', 2**(s+s_)/(1.0-2**(-s-s_)))

(s, r) = (1.0, 0.0)
Optimization terminated successfully.
         Current function value: -2.000000
         Iterations: 2
         Function evaluations: 41372
Max ratio:  1.9999999999994613
Theoretical bound:  4.0
(s, r) = (2.0, 1.0)
Optimization terminated successfully.
         Current function value: -8.000000
         Iterations: 2
         Function evaluations: 37921
Max ratio:  7.999999999999994
Theoretical bound:  9.142857142857142
(s, r) = (2.5, 1.5)
Optimization terminated successfully.
         Current function value: -16.000000
         Iterations: 2
         Function evaluations: 29570
Max ratio:  15.999999999767285
Theoretical bound:  17.066666666666666
(s, r) = (4.0, 3.0)
Optimization terminated successfully.
         Current function value: -128.000000
         Iterations: 2
         Function evaluations: 31272
Max ratio:  127.99999999999993
Theoretical bound:  129.00787401574803


$$
\|au\|_{W^s}
\leq \frac{3-2^{-s}}{(1-2^{-s})^2}\cdot2^{s}\|a\|_{W^s}\|u\|_{W^s}
$$

In [22]:
# test with optimization

np.random.seed(0)
for s in [1.0, 2.0, 2.5, 4.0]:
    print('s = ', s)
    x0 = np.random.randn(514)

    def F_obj(x):
        a = map_freq(para.N, 128, x[:257])
        u = map_freq(para.N, 128, x[257:])
        pd = para.product(a, u)
        r = para.Ws_norm(pd, s) / para.Ws_norm(a, s) / para.Ws_norm(u, s)
        return -r

    from scipy.optimize import minimize
    res = minimize(
        F_obj,
        x0,
        method="Powell",                # or "Nelder-Mead"
        options={"maxiter": 10000, "maxfev":200000, "disp": True}
    )
    print('Max ratio: ', -res.fun)
    print('Theoretical bound: ', (3-2**(-s)) * 2**s / (1.0-2**(-s))**2)

s =  1.0
Optimization terminated successfully.
         Current function value: -5.254991
         Iterations: 5
         Function evaluations: 66592
Max ratio:  5.254990544872603
Theoretical bound:  20.0
s =  2.0
Optimization terminated successfully.
         Current function value: -2.666137
         Iterations: 3
         Function evaluations: 18619
Max ratio:  2.666136545780808
Theoretical bound:  19.555555555555557
s =  2.5
Optimization terminated successfully.
         Current function value: -4.960330
         Iterations: 3
         Function evaluations: 28573
Max ratio:  4.960329877214558
Theoretical bound:  23.565947544666333
s =  4.0
Optimization terminated successfully.
         Current function value: -8.533211
         Iterations: 3
         Function evaluations: 29643
Max ratio:  8.533210752659482
Theoretical bound:  53.47555555555556


$$
\big\|(T_aT_b-T_{ab})u\big\|_{W^{s+r}}
\leq 
K_{{\rm CM};r,s}\|a\|_{W^r}\|b\|_{W^r}\|u\|_{W^s},
$$
where
$$
K_{{\rm CM};r,s}
=\frac{2}{(1-2^{-r})(1-2^{-s})(1-2^{-(r+s)})}\cdot 2^{r+2s}
+\frac{3+2^{-r}}{(1-2^{-r})^2}\cdot 2^{2r+s}.
$$

In [25]:
# test with optimization

np.random.seed(0)
for s in [1.5, 2.5, 3.0, 4.0]:
    s_ = s - 1
    print(f'(s, r) = ({s}, {s_})')
    x0 = np.random.randn(129*3)

    def F_obj(x):
        a = map_freq(para.N, 64, x[:129])
        b = map_freq(para.N, 64, x[129:258])
        u = map_freq(para.N, 64, x[258:])
        Tb_u = para.paraproduct(b, u)
        Ta_Tb_u = para.paraproduct(a, Tb_u)
        Tab_u = para.paraproduct(para.product(a, b), u)
        R_cm = Ta_Tb_u - Tab_u
        r = para.Ws_norm(R_cm, s+s_) / para.Ws_norm(a, s_) / para.Ws_norm(b, s_) / para.Ws_norm(u, s)
        return -r

    from scipy.optimize import minimize
    res = minimize(
        F_obj,
        x0,
        method="Powell",                # or "Nelder-Mead"
        options={"maxiter": 10000, "maxfev":100000, "disp": True}
    )
    print('Max ratio: ', -res.fun)
    K = 2 / (1.0-2**(-s_)) / (1.0-2**(-s)) / (1.0-2**(-s-s_)) * 2**(s_+2*s)
    K += (3+2**(-s_)) / (1-2**(-s_))**2 * 2**(2*s_+s)
    print('Theoretical bound: ', K)

(s, r) = (1.5, 0.5)
Optimization terminated successfully.
         Current function value: -4.999838
         Iterations: 4
         Function evaluations: 48638
Max ratio:  4.999837545177965
Theoretical bound:  403.7933344334783
(s, r) = (2.5, 1.5)
Optimization terminated successfully.
         Current function value: -6.911794
         Iterations: 4
         Function evaluations: 45339
Max ratio:  6.911793605426819
Theoretical bound:  725.9958983055103
(s, r) = (3.0, 2.0)
Optimization terminated successfully.
         Current function value: -0.250000
         Iterations: 2
         Function evaluations: 12447
Max ratio:  0.24999999999999942
Theoretical bound:  1544.9134664618537
(s, r) = (4.0, 3.0)
Optimization terminated successfully.
         Current function value: -324.472312
         Iterations: 2
         Function evaluations: 18288
Max ratio:  324.4723116297417
Theoretical bound:  9212.127569768065


In [15]:
# test with optimization

np.random.seed(0)
for s in [1.5, 2.5, 3.0, 4.0]:
    s_ = s - 1
    print(f'(s, r) = ({s}, {s_})')
    x0 = np.random.randn(129*2+257)

    def F_obj(x):
        a = map_freq(para.N, 64, x[:129])
        b = map_freq(para.N, 64, x[129:258])
        u = map_freq(para.N, 128, x[258:])
        Tb_u = para.paraproduct(b, u)
        Ta_Tb_u = para.paraproduct(a, Tb_u)
        Tab_u = para.paraproduct(para.product(a, b), u)
        R_cm = Ta_Tb_u - Tab_u
        r = para.Ws_norm(R_cm, s+s_) / para.Ws_norm(a, s_) / para.Ws_norm(b, s_) / para.Ws_norm(u, s)
        return -r

    from scipy.optimize import minimize
    res = minimize(
        F_obj,
        x0,
        method="Powell",                # or "Nelder-Mead"
        options={"maxiter": 10000, "maxfev":100000, "disp": True}
    )
    print('Max ratio: ', -res.fun)
    K = 2 / (1.0-2**(-s_)) / (1.0-2**(-s)) / (1.0-2**(-s-s_)) * 2**(s_+2*s)
    K += (3+2**(-s_)) / (1-2**(-s_))**2 * 2**(2*s_+s)
    print('Theoretical bound: ', K)

-28.952380952380963
